# 将Bokeh的图表嵌入到ipywidget控件中

本节介绍如何将Bokeh的图表嵌入到ipywidget的控件界面中，并实现Python和Javascript之间双向通信。

In [1]:
from bokehelp import output_notebook, show_figure
from bokeh.io import output_notebook #如果希望从CDN载入bokeh.js，则取消该行注释
from bokeh.plotting import Figure
from bokeh.models import ColumnDataSource

output_notebook()

Loading BokehJS ...

使用`bokeh.embed.notebook_div()`将图表对象转换为HTML代码，然后将HTML代码用`ipywidgets.HTML`显示即可实现图表的嵌入。不过为了实现双向通信，还需要进一步的处理。

下面的`BokehFigure`类从`ipywidgets.HTML`继承，其`value`属性即为图表对应的HTML代码。

❶我们需要模拟`bokeh.io.show()`显示图表的功能，这样才能使用`push_notebook()`实现Python到Javascript的通信。本段代码仿照`bokeh.embed`中显示图表的程序。Python->Javascript的通信由`_CommsHandle`对象实现，由于该对象必须在图表显示之后运行，因此这里覆盖基类的`_ipython_display_`方法，在调用` super()._ipython_display_(**kwargs)`之后，❹创建通信用的`_CommsHandle`对象。调用Bokeh提供的`push_notebook()`函数更新图表时，将`Document`、`State`和`_CommsHandle`等对象作为参数传递给它即可实现被嵌入图表的更新。

❷调用`notebook_div()`生成图表的HTML之后，添加实现Javascript->Python通信的代码。它为Javascript中的`Figure`对象添加`send_data()`方法。然后通过`comm_manager.register_target()`添加Python端的通信处理函数。当在Javascript中调用`Figure.send_data()`发送信息时，将调用`on_recv()`方法，在该方法中会调用用户提供的回调函数`callback`。

In [2]:
from bokeh.core.state import State
from bokeh.document import Document
from bokeh.io import _CommsHandle, push_notebook
from bokeh.embed import notebook_div
from bokeh.util.notebook import get_comms
from IPython import get_ipython
from ipywidgets import HTML
from jinja2 import Template

JSCode = Template("""
<script>
(function(){
    var fig = Bokeh.index['{{fid}}'];
    fig.model.send_data = function(data){
        var comm = Jupyter.notebook.kernel.comm_manager.new_comm('{{fid}}', data);
        comm.close();
    };
})();
</script>
""")

class BokehFigure(HTML):
    def __init__(self, fig, callback=None, **kwargs):
        super().__init__(**kwargs)
        self.callback = callback

        #❶ 本段代码参考Bokeh中显示图表的程序
        self.state = State()
        self.doc = Document()
        self.state._document = self.doc
        self.state.output_notebook = True
        self.fig = fig
        self.doc.add_root(self.fig)
        
        #❷ 生成HTML代码，并添加实现Javascript->Python通信的代码
        self.comms_target = self.fig._id
        self.value = notebook_div(fig, self.comms_target) 
        self.value += JSCode.render(fid=self.fig._id)
        comm_manager = get_ipython().kernel.comm_manager
        comm_manager.register_target(self.fig._id, self.on_recv)
        
        self._handle = None
        
    def on_recv(self, comm, data):
        if self.callback is not None:
            self.callback(data["content"]["data"]) 
        
    @property
    def handle(self): #❸
        if self._handle is not None:
            return self._handle
        self._handle = _CommsHandle(get_comms(self.comms_target), 
                                   self.state.document, 
                                   self.state.document.to_json())
        self.state.last_comms_handle = self._handle
        return self._handle
    
    def __del__(self):
        self.handle.comms.close()
    
    def _ipython_display_(self, **kwargs):
        super()._ipython_display_(**kwargs)
        _ = self.handle  #❹
        
    def push_notebook(self):
        push_notebook(self.doc, self.state, self.handle) #❺

In [3]:
Figure(x_range=[-2, 3])

In [4]:
import numpy as np
from bokeh.models import CustomJS
from ipywidgets import FloatSlider, VBox, HBox, Textarea

N = 100
src_data = np.random.randn(N, 2)
r = 1.4
theta = np.linspace(0, 2*np.pi, N)
dst_data = np.c_[r * np.sin(2*theta), r * np.cos(theta)]
color = ["#{:02x}{:02x}{:02x}".format(*row) for row in np.random.randint(0, 200, (N, 3))]

fig = Figure(tools="lasso_select", x_range=[-3, 3], y_range=[-3, 3], 
             plot_width=350, plot_height=350)
source = ColumnDataSource(data={"x":src_data[:, 0], "y":dst_data[:, 1], "color":color})
source.callback = CustomJS(args=dict(fig=fig), code="""
    fig.send_data(cb_obj.get('selected')['1d'].indices);
    """)
fig.circle(x="x", y="y", color="color", size=10, source=source)

selected_info = Textarea(width=300)

def on_select(data):
    selected_info.value = str(data)
    
bokeh_fig = BokehFigure(fig, callback=on_select)
w_slider = FloatSlider(min=0, max=1, step=0.01, value=0, description="k:", width=250)

def f(evt):
    k = evt["new"]
    new_data = (1 - k) * src_data + k * dst_data
    source.data["x"] = new_data[:, 0]
    source.data["y"] = new_data[:, 1]
    bokeh_fig.push_notebook()

w_slider.observe(f, names=["value"])
box = HBox([selected_info, VBox([w_slider, bokeh_fig])])
box